# Images of satellites classification

In [ ]:
from google_images_download import google_images_download   #importing the library
import pandas
import cv2 #to load and displays images
import matplotlib.pyplot as plt
import imutils

## Download images of satellites / non satellites

In [ ]:
nb_images = 2000 # The number of images to download (both from satellite and non satellite images)
download = False  # Are we doanloading the images ?
chromeDriverPath = "/usr/bin/chromedriver"

In [ ]:
def downloadImages(tags, nbImgs):
    response = google_images_download.googleimagesdownload()   #class instantiation

    arguments = {"keywords":','.join(tags),"limit":nbImgs,"print_urls":False, "extract_metadata":True, "thumbnail_only":True, "chromedriver":chromeDriverPath}   #creating list of arguments
    if download:
        response.download(arguments)   #passing the arguments to the function

download list of satellites categories

In [ ]:
# categories taken from https://www.omicsonline.org/conferences-list/types-of-satellites-and-applications
categories = ["Communications Satellite","Remote Sensing Satellite","Navigation Satellite","LEO satellite", 
              "MEO satellite", "HEO satellite","GPS satellite","GEO satellite","Drone Satellite",
              "Polar Satellite","Nano Satellites","CubeSats","SmallSats"]

In [ ]:
downloadImages(categories, nb_images)

In [ ]:
catpd = [pandas.read_json("logs/" + str(cat) + ".json") for cat in categories]

### download of images

In [ ]:
downloadImages(["satellite","images"], nb_images)

In [ ]:
imgpd = pandas.read_json("logs/images.json")
satpd = pandas.read_json("logs/satellite.json")

In [ ]:
imgpd.image_filename.count(), satpd.image_filename.count()

In [ ]:
satpd.iloc[0]

### display of the first images

In [ ]:
def draw_imgs(imgs_list):
    l = int(len(imgs_list))
    n = int(len(imgs_list[0]))
    _, axs = plt.subplots(l, n, figsize=(15, 15))
    axs = axs.flatten()
    for img, ax in zip([item for sublist in imgs_list for item in sublist], axs):
        ax.imshow(img)
    plt.show()

In [ ]:
listnonsat = []
for i in range(0,7):
    img =cv2.imread("./downloads/images - thumbnail/"+imgpd.image_filename[i])
    img = cv2.resize(img, (244,244), cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    listnonsat.append(img)
    
listsat = []
for i in range(0,7):
    img =cv2.imread("./downloads/satellite - thumbnail/"+satpd.image_filename[i])
    img = cv2.resize(img, (244,244), cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    listsat.append(img)
    
draw_imgs([listnonsat, listsat])



In [ ]:
listcat = []
for current,cat in zip(catpd, categories):
    print(cat)
    listcurrent = []
    for i in range(0,7):
        img =cv2.imread("./downloads/"+str(cat)+" - thumbnail/"+current.image_filename[i])
        img = cv2.resize(img, (244,244), cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        listcurrent.append(img)
    listcat.append(listcurrent)
    
draw_imgs(listcat)

## Classify satellite images

In [ ]:
listnonsat = []
for i in range(0,imgpd.shape[0]):
    try:
        img =cv2.imread("./downloads/images - thumbnail/"+imgpd.image_filename[i])
        img = cv2.resize(img, (244,244), cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        listnonsat.append(img)
    except:
        print("image not resizable")
    
listsat = []
for i in range(0,satpd.shape[0]):
    try:
        img =cv2.imread("./downloads/satellite - thumbnail/"+satpd.image_filename[i])
        img = cv2.resize(img, (244,244), cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
        listsat.append(img)
    except:
        print("image not resizable")
    

In [ ]:
len(listnonsat), len(listsat)

In [ ]:
from keras.models import Sequential

model = Sequential()